# GKtau

In [15]:
import pandas as pd
from src.gktau import *

In [16]:
data = pd.read_csv('smog_data_cleaned.csv')
data = data.drop(columns=[ 
                        "Engine Size(L)",
                        "CO2 Emissions(g/km)",
                        "Fuel Consumption (City (L/100 km)",
                        "Fuel Consumption(Hwy (L/100 km))",
                        "Fuel Consumption(Comb (L/100 km))",
                        "Fuel Consumption(Comb (mpg))"])
summary_dup = data.loc[:,~data.columns.isin(['Cylinders', 'NumberOfGears','Smog Rating'])].describe()
summary_dup

,Log Fuel Consumption(Comb (L/100 km)),Log CO2 Emissions(g/km),Log Engine Size(L),Log Fuel Consumption(Hwy (L/100 km)),Log Fuel Consumption(City (L/100 km))
count,946.000000,946.000000,946.000000,946.000000,946.000000
mean,2.372682,5.525874,1.077242,2.207952,2.487601
std,0.262099,0.255839,0.409394,0.240453,0.283543
min,1.386294,4.543295,0.182322,1.360977,1.386294
25%,2.208274,5.362463,0.693147,2.041220,2.322388
50%,2.379546,5.549076,1.098612,2.219203,2.501436
75%,2.557227,5.706278,1.335001,2.370244,2.687847
max,3.261935,6.410175,2.079442,3.039749,3.411148


In [17]:
# Only run this cell once! If want to rerun it, please rerun the entire jupyter file.

bins_dict = {}
mins = summary_dup.loc['min']
maxs = summary_dup.loc['max']
ind = 0
for col in summary_dup.columns:
  stride = (maxs[ind]-mins[ind])/4
  bins_dict[col] = [[mins[ind], mins[ind]+stride, mins[ind]+2*stride, mins[ind]+3*stride, maxs[ind]+1]]
  bins_dict[col].append([col+"1", col+"2", col+"3", col+"4"])
  ind = ind + 1
for col in summary_dup.columns:
  bins = bins_dict[col][0]
  labels = bins_dict[col][1]
  data[col] = pd.cut(data[col], bins=bins, labels=labels)

In [19]:
gktau_dict = {}
for col in data:
  if (col != 'Smog Rating' and col != 'Log Fuel Consumption(Comb (L/100 km))'):
    table = pd.crosstab(data[col], data['Smog Rating'])
    gk = GKtau(table, iprop=False)
    gktau_dict[col] = str(round(math.sqrt(gk), 3))
gktau_df = pd.DataFrame.from_dict(gktau_dict, orient='index', columns=['Smog Rating Correlation'])
display(gktau_df)

,Smog Rating Correlation
Make,0.53
Vehicle Class,0.269
Cylinders,0.332
Transmission,0.304
Fuel Type,0.204
AllOrFourWheelDrive,0.139
NumberOfGears,0.326
isCVT,0.178
Log CO2 Emissions(g/km),0.306
Log Engine Size(L),0.267
